In [2]:
!pip install xlrd==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 103 kB 7.7 MB/s 
  Attempting uninstall: xlrd
    Found existing installation: xlrd 1.1.0
    Uninstalling xlrd-1.1.0:
      Successfully uninstalled xlrd-1.1.0


In [3]:
!pip install stop_words

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for stop-words: filename=stop_words-2018.7.23-py3-none-any.whl size=32912 sha256=f4bee5a6b7cacee4b9bad29257b3ff96217aa3f2a003881f0290b454260442f9
  Stored in directory: /root/.cache/pip/wheels/fb/86/b2/277b10b1ce9f73ce15059bf6975d4547cc4ec3feeb651978e9
Successfully built stop-words


In [4]:
!pip install pymorphy2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 1.8 MB/s 
     |████████████████████████████████| 8.2 MB 11.0 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=223a9087d111050a5ae8689e8e745d0bc7ec22584fd1abfc076650e9fe7f582d
  Stored in directory: /root/.cache/pip/wheels/72/b0/3f/1d95f96ff986c7dfffe46ce2be4062f38ebd04b506c77c81b9
Successfully built docopt


In [5]:
!pip install tensorflow tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 7.2 MB/s 


In [6]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
import pandas as pd
import numpy as np
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [8]:
df = pd.read_excel('/content/drive/MyDrive/NLP/reviews.xls')

In [9]:
df.head(10)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,У меня работает все четко. В отличии от банком...,2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [10]:
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), [int(.6*len(df)), int(.8*len(df))])

In [11]:
df_train.head()

,Rating,Content,Date
1457,5,Всё отлично!!!,2017-08-11
19916,5,Мне нравится. Удобно. Еще ни разу не подвел!,2017-06-19
2112,3,Иногда слишком долго приходится ждать пока зап...,2017-08-10
15154,5,Отличное приложение. Легко отслеживать свои ра...,2017-07-22
18329,5,Все грамотно,2017-07-06


In [12]:
df_test.head()

,Rating,Content,Date
12225,3,Всё было хорошо. Редкие проблемы со входом - п...,2017-07-27
1051,1,"Не знаю, что произошло, но приложение стало за...",2017-08-12
14519,5,Замечательное приложение. Решает многие вопрос...,2017-07-23
11410,4,Долго запускается. А так норм.,2017-07-28
9221,1,"Тормозит, не регистрируется ссылаясь на безопа...",2017-08-01


In [13]:
df_val.head()

,Rating,Content,Date
20015,2,После обновления постоянно пишет что прошивка ...,2017-06-17
9828,5,"Хорошее приложение,жаль нет переводов на карты...",2017-07-31
938,1,Было отличное приложение. Потом начал заходить...,2017-08-12
14457,3,Штрафы не оплатить,2017-07-23
395,5,Супер мне нравиться очень и быстро,2017-08-13


In [14]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)
df_test['Content'] = df_test['Content'].apply(preprocess_text)

In [15]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [16]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [18]:
max_words = 200
max_len = 40
num_classes = 1

# Training
epochs = 20
batch_size = 512
print_batch_n = 100

In [19]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [20]:
tokens_filtered_top[:10]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный',
 'телефон',
 'супер']

In [21]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [22]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [23]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in df_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)

In [24]:
x_train[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 2], dtype=int32)

In [25]:
import tensorflow as tf
import tensorflow_addons as tfa

In [26]:
import keras

In [27]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  

In [28]:
num_classes = 6
y_train = keras.utils.to_categorical(df_train["Rating"], num_classes)
y_val = keras.utils.to_categorical(df_val["Rating"], num_classes)

In [29]:
metrics = {'model': [],
           'test_score': [],
           'test_accuracy': []}
def metrics_append(model, score, accuracy):
    metrics['model'].append(model)
    metrics['test_score'].append(score)
    metrics['test_accuracy'].append(accuracy)

In [30]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [31]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [32]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')  

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/20
22/22 [==============================] - 4s 157ms/step - loss: 1.7419 - accuracy: 0.0403 - val_loss: 1.6849 - val_accuracy: 0.0379
Epoch 2/20
22/22 [==============================] - 5s 227ms/step - loss: 1.6457 - accuracy: 0.0437 - val_loss: 1.5877 - val_accuracy: 0.0694


In [33]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 0s 34ms/step - loss: 1.5901 - accuracy: 0.0704


Test score: 1.59005868434906
Test accuracy: 0.07042594254016876


In [34]:
metrics_append('CNN', score[0], score[1])

**RNN**

In [35]:
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D, SimpleRNN, LSTM, GRU, Masking

In [36]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True))

model.add(SimpleRNN(128))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [37]:
early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
22/22 [==============================] - 5s 152ms/step - loss: 1.1874 - accuracy: 0.6235 - val_loss: 0.8607 - val_accuracy: 0.7242
Epoch 2/20
22/22 [==============================] - 3s 135ms/step - loss: 0.9033 - accuracy: 0.7240 - val_loss: 0.7817 - val_accuracy: 0.7444


In [38]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 0s 41ms/step - loss: 0.8113 - accuracy: 0.7323


Test score: 0.8112689852714539
Test accuracy: 0.7323330044746399


In [39]:
metrics_append('RNN', score[0], score[1])

**LSTM**

In [40]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True))

model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
22/22 [==============================] - 17s 632ms/step - loss: 1.3009 - accuracy: 0.6625 - val_loss: 0.9187 - val_accuracy: 0.7202
Epoch 2/20
22/22 [==============================] - 13s 614ms/step - loss: 0.9655 - accuracy: 0.7127 - val_loss: 0.8159 - val_accuracy: 0.7395


In [41]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 1s 137ms/step - loss: 0.8476 - accuracy: 0.7227


Test score: 0.8476260900497437
Test accuracy: 0.7226524949073792


In [42]:
metrics_append('LSTM', score[0], score[1])

**GRU**

In [43]:
model = Sequential()

model.add(
    Embedding(input_dim=max_words,
              input_length=max_len,
              output_dim=128,
              trainable=True))

model.add(GRU(128, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
22/22 [==============================] - 17s 623ms/step - loss: 1.2606 - accuracy: 0.6829 - val_loss: 0.8935 - val_accuracy: 0.7202
Epoch 2/20
22/22 [==============================] - 12s 568ms/step - loss: 0.8776 - accuracy: 0.7127 - val_loss: 0.7779 - val_accuracy: 0.7331


In [44]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 1s 110ms/step - loss: 0.8138 - accuracy: 0.7164


Test score: 0.8137953281402588
Test accuracy: 0.7163600921630859


In [45]:
metrics_append('GRU', score[0], score[1])

In [46]:
pd.DataFrame(metrics).sort_values('test_accuracy', ascending=False)

,model,test_score,test_accuracy
1,RNN,0.811269,0.732333
2,LSTM,0.847626,0.722652
3,GRU,0.813795,0.716360
0,CNN,1.590059,0.070426


**CNN_LSTM**

In [47]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dense(64, activation='relu'))
model.add(Dense(num_classes))
model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

early_stopping=EarlyStopping(monitor='val_loss')  


history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
22/22 [==============================] - 19s 728ms/step - loss: 1.3255 - accuracy: 0.7070 - val_loss: 1.0303 - val_accuracy: 0.7202
Epoch 2/20
22/22 [==============================] - 17s 793ms/step - loss: 0.9141 - accuracy: 0.7251 - val_loss: 0.7927 - val_accuracy: 0.7524


In [48]:
score = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=1)
print('\n')
print('Test score:', score[0])
print('Test accuracy:', score[1])

9/9 [==============================] - 1s 161ms/step - loss: 0.8221 - accuracy: 0.7408


Test score: 0.822081446647644
Test accuracy: 0.7408034801483154


In [49]:
metrics_append('CNN_LSTM', score[0], score[1])

In [50]:
pd.DataFrame(metrics).sort_values('test_accuracy', ascending=False)

,model,test_score,test_accuracy
4,CNN_LSTM,0.822081,0.740803
1,RNN,0.811269,0.732333
2,LSTM,0.847626,0.722652
3,GRU,0.813795,0.716360
0,CNN,1.590059,0.070426
